In [1]:
import os
# Bertsum directory chdir
os.chdir('./src')

In [2]:
%ls

__pycache__/    models/  prepro/        requirements.txt  train.py
distributed.py  others/  preprocess.py  tmp/


## 추출요약 코드

In [314]:
"""
    Main training workflow
"""
from __future__ import division

import argparse
import glob
import os
import random
import signal
import time
import numpy as np

import torch
from pytorch_pretrained_bert import BertConfig


import distributed
from models import data_loader, model_builder
from models.data_loader import load_dataset
from models.model_builder import Summarizer
from tensorboardX import SummaryWriter
from models.reporter import ReportMgr
from models.stats import Statistics
from others.logging import logger
# from models.trainer import build_trainer
# build_trainer의 dependency package pyrouge.utils가 import되지 않아 직접 셀에 삽입
from others.logging import logger, init_logger
import easydict

args = easydict.EasyDict({
    "encoder":'classifier',
    "mode":'summary',
    "bert_data_path":'../bert_sample/korean',
    "model_path":'../models/bert_classifier',
    "bert_model":'../001_bert_morp_pytorch',
    "result_path":'../results/korean',
    "temp_dir":'.',
    "bert_config_path":'../001_bert_morp_pytorch/bert_config.json',
    "batch_size":1000,
    "use_interval":True,
    "hidden_size":128,
    "ff_size":512,
    "heads":4,
    "inter_layers":2,
    "rnn_size":512,
    "param_init":0,
    "param_init_glorot":True,
    "dropout":0.1,
    "optim":'adam',
    "lr":2e-3,
    "report_every":1,
    "save_checkpoint_steps":5,
    "block_trigram":True,
    "recall_eval":False,
    
    "accum_count":1,
    "world_size":1,
    "visible_gpus":'-1',
    "gpu_ranks":'0',
    "log_file":'../logs/bert_classifier',
    "test_from":'../models/bert_classifier2/model_step_35000.pt'
})


def build_trainer(args, device_id, model,
                  optim):
    """
    Simplify `Trainer` creation based on user `opt`s*
    Args:
        opt (:obj:`Namespace`): user options (usually from argument parsing)
        model (:obj:`onmt.models.NMTModel`): the model to train
        fields (dict): dict of fields
        optim (:obj:`onmt.utils.Optimizer`): optimizer used during training
        data_type (str): string describing the type of data
            e.g. "text", "img", "audio"
        model_saver(:obj:`onmt.models.ModelSaverBase`): the utility object
            used to save the model
    """
    device = "cpu" if args.visible_gpus == '-1' else "cuda"


    grad_accum_count = args.accum_count
    n_gpu = args.world_size

    if device_id >= 0:
        gpu_rank = int(args.gpu_ranks[device_id])
    else:
        gpu_rank = 0
        n_gpu = 0

    #print('gpu_rank %d' % gpu_rank)

    tensorboard_log_dir = args.model_path

    writer = SummaryWriter(tensorboard_log_dir, comment="Unmt")

    report_manager = ReportMgr(args.report_every, start_time=-1, tensorboard_writer=writer)

    trainer = Trainer(args, model, optim, grad_accum_count, n_gpu, gpu_rank, report_manager)

    # print(tr)
    if (model):
        n_params = _tally_parameters(model)
        #logger.info('* number of parameters: %d' % n_params)

    return trainer

class Trainer(object):
    """
    Class that controls the training process.

    Args:
            model(:py:class:`onmt.models.model.NMTModel`): translation model
                to train
            train_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            valid_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            optim(:obj:`onmt.utils.optimizers.Optimizer`):
               the optimizer responsible for update
            trunc_size(int): length of truncated back propagation through time
            shard_size(int): compute loss in shards of this size for efficiency
            data_type(string): type of the source input: [text|img|audio]
            norm_method(string): normalization methods: [sents|tokens]
            grad_accum_count(int): accumulate gradients this many times.
            report_manager(:obj:`onmt.utils.ReportMgrBase`):
                the object that creates reports, or None
            model_saver(:obj:`onmt.models.ModelSaverBase`): the saver is
                used to save a checkpoint.
                Thus nothing will be saved if this parameter is None
    """

    def __init__(self,  args, model,  optim,
                  grad_accum_count=1, n_gpu=1, gpu_rank=1,
                  report_manager=None):
        # Basic attributes.
        self.args = args
        self.save_checkpoint_steps = args.save_checkpoint_steps
        self.model = model
        self.optim = optim
        self.grad_accum_count = grad_accum_count
        self.n_gpu = n_gpu
        self.gpu_rank = gpu_rank
        self.report_manager = report_manager

        self.loss = torch.nn.BCELoss(reduction='none')
        assert grad_accum_count > 0
        # Set model in training mode.
        if (model):
            self.model.train()

    def summary(self, test_iter, step, cal_lead=False, cal_oracle=False):
        """ Validate model.
            valid_iter: validate data iterator
        Returns:
            :obj:`nmt.Statistics`: validation loss statistics
        """
        # Set model in validating mode.
        def _get_ngrams(n, text):
            ngram_set = set()
            text_length = len(text)
            max_index_ngram_start = text_length - n
            for i in range(max_index_ngram_start + 1):
                ngram_set.add(tuple(text[i:i + n]))
            return ngram_set

        def _block_tri(c, p):
            tri_c = _get_ngrams(3, c.split())
            for s in p:
                tri_s = _get_ngrams(3, s.split())
                if len(tri_c.intersection(tri_s))>0:
                    return True
            return False

        if (not cal_lead and not cal_oracle):
            self.model.eval()
        stats = Statistics()

        
        with torch.no_grad():
            for batch in test_iter:
                src = batch.src
                labels = batch.labels
                segs = batch.segs
                clss = batch.clss
                mask = batch.mask
                mask_cls = batch.mask_cls


                gold = []
                pred = []

                if (cal_lead):
                    selected_ids = [list(range(batch.clss.size(1)))] * batch.batch_size
                elif (cal_oracle):
                    selected_ids = [[j for j in range(batch.clss.size(1)) if labels[i][j] == 1] for i in
                                    range(batch.batch_size)]
                else:
                    sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

                    # loss = self.loss(sent_scores, labels.float())
                    # loss = (loss * mask.float()).sum()
                    # batch_stats = Statistics(float(loss.cpu().data.numpy()), len(labels))
                    # stats.update(batch_stats)

                    sent_scores = sent_scores + mask.float()
                    sent_scores = sent_scores.cpu().data.numpy()
                    selected_ids = np.argsort(-sent_scores, 1)
                # selected_ids = np.sort(selected_ids,1)
                

        return selected_ids


    def _gradient_accumulation(self, true_batchs, normalization, total_stats,
                               report_stats):
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            if self.grad_accum_count == 1:
                self.model.zero_grad()

            src = batch.src
            labels = batch.labels
            segs = batch.segs
            clss = batch.clss
            mask = batch.mask
            mask_cls = batch.mask_cls

            sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

            loss = self.loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()
            (loss/loss.numel()).backward()
            # loss.div(float(normalization)).backward()

            batch_stats = Statistics(float(loss.cpu().data.numpy()), normalization)


            total_stats.update(batch_stats)
            report_stats.update(batch_stats)

            # 4. Update the parameters and statistics.
            if self.grad_accum_count == 1:
                # Multi GPU gradient gather
                if self.n_gpu > 1:
                    grads = [p.grad.data for p in self.model.parameters()
                             if p.requires_grad
                             and p.grad is not None]
                    distributed.all_reduce_and_rescale_tensors(
                        grads, float(1))
                self.optim.step()

        # in case of multi step gradient accumulation,
        # update only after accum batches
        if self.grad_accum_count > 1:
            if self.n_gpu > 1:
                grads = [p.grad.data for p in self.model.parameters()
                         if p.requires_grad
                         and p.grad is not None]
                distributed.all_reduce_and_rescale_tensors(
                    grads, float(1))
            self.optim.step()

    def _save(self, step):
        real_model = self.model
        # real_generator = (self.generator.module
        #                   if isinstance(self.generator, torch.nn.DataParallel)
        #                   else self.generator)

        model_state_dict = real_model.state_dict()
        # generator_state_dict = real_generator.state_dict()
        checkpoint = {
            'model': model_state_dict,
            # 'generator': generator_state_dict,
            'opt': self.args,
            'optim': self.optim,
        }
        checkpoint_path = os.path.join(self.args.model_path, 'model_step_%d.pt' % step)
        #logger.info("Saving checkpoint %s" % checkpoint_path)
        # checkpoint_path = '%s_step_%d.pt' % (FLAGS.model_path, step)
        if (not os.path.exists(checkpoint_path)):
            torch.save(checkpoint, checkpoint_path)
            return checkpoint, checkpoint_path

    def _start_report_manager(self, start_time=None):
        """
        Simple function to start report manager (if any)
        """
        if self.report_manager is not None:
            if start_time is None:
                self.report_manager.start()
            else:
                self.report_manager.start_time = start_time

    def _maybe_gather_stats(self, stat):
        """
        Gather statistics in multi-processes cases

        Args:
            stat(:obj:onmt.utils.Statistics): a Statistics object to gather
                or None (it returns None in this case)

        Returns:
            stat: the updated (or unchanged) stat object
        """
        if stat is not None and self.n_gpu > 1:
            return Statistics.all_gather_stats(stat)
        return stat

    def _maybe_report_training(self, step, num_steps, learning_rate,
                               report_stats):
        """
        Simple function to report training stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_training` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_training(
                step, num_steps, learning_rate, report_stats,
                multigpu=self.n_gpu > 1)

    def _report_step(self, learning_rate, step, train_stats=None,
                     valid_stats=None):
        """
        Simple function to report stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_step` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_step(
                learning_rate, step, train_stats=train_stats,
                valid_stats=valid_stats)

    def _maybe_save(self, step):
        """
        Save the model if a model saver is set
        """
        if self.model_saver is not None:
            self.model_saver.maybe_save(step)

            
def summary(args, b_list, device_id, pt, step, model):
    device_id = 0
    device = "cpu" if args.visible_gpus == '-1' else "cuda"
    if (pt != ''):
        test_from = pt
    else:
        test_from = args.test_from
    
    opt = vars(checkpoint['opt'])
    for k in opt.keys():
        if (k in model_flags):
            setattr(args, k, opt[k])
    #print(args)

    #model.load_cp(checkpoint)
    #model.eval()

    test_iter =data_loader.Dataloader(args, _lazy_dataset_loader(b_list),
                                  args.batch_size, device,
                                  shuffle=False, is_test=True)
    trainer = build_trainer(args, device_id, model, None)
    result = trainer.summary(test_iter,step)
    return result

def _tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

args.gpu_ranks = [int(i) for i in args.gpu_ranks.split(',')]
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

init_logger(args.log_file)
device = "cpu" if args.visible_gpus == '-1' else "cuda"
device_id = 0 if device == "cuda" else -1
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']

In [315]:
import argparse
import json
import os
import time
import urllib3
from glob import glob
import collections
import six
import gc
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.utils import download as _download

def do_lang ( openapi_key, text ) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    requestJson = {  
        "argument": {
            "text": text,
            "analysis_code": "morp"
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  openapi_key},
        body=json.dumps(requestJson)
    )

    json_data = json.loads(response.data.decode('utf-8'))
    json_result = json_data["result"]
    
    if json_result == -1:
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason
    else:
        json_data = json.loads(response.data.decode('utf-8'))
    
        json_return_obj = json_data["return_object"]
        
        return_result = ""
        json_sentence = json_return_obj["sentence"]
        for json_morp in json_sentence:
            for morp in json_morp["morp"]:
                return_result = return_result+str(morp["lemma"])+"/"+str(morp["type"])+" "

        return return_result

def get_kobert_vocab(cachedir="./tmp/"):
    # Add BOS,EOS vocab
    tokenizer = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/tokenizers/kobert_news_wiki_ko_cased-1087f8699e.spiece",
        #"fname": "/opt/ml/SumAI/bertsum/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece",
        "chksum": "ae5711deb3",
    }
    
    vocab_info = tokenizer
    vocab_file = _download(
        #vocab_info["url"], vocab_info["fname"], vocab_info["chksum"], cachedir=cachedir
        vocab_info["url"], vocab_info["chksum"], cachedir=cachedir
    )

    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
        vocab_file[0], padding_token="[PAD]", bos_token="[BOS]", eos_token="[EOS]"
    )
    return vocab_b_obj

    
class BertData():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        #self.tokenizer = Tokenizer(vocab_file_path)
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [''.join(s) for s in src]


        idxs = [i for i, s in enumerate(src) if (len(s) > 0)]

        src = [src[i][:20000] for i in idxs]
        src = src[:10000]

        if (len(src) < 3):
            return None

        src_txt = [''.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = text.split(' ')
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        tgt_txt = None
        src_txt = [original_src_txt[i] for i in idxs]
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt
    
def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")
        
class Tokenizer(object):
    
    def __init__(self, vocab_file_path):
        self.vocab_file_path = vocab_file_path
        """Loads a vocabulary file into a dictionary."""
        vocab = collections.OrderedDict()
        index = 0
        with open(self.vocab_file_path, "r", encoding='utf-8') as reader:

            while True:
                token = convert_to_unicode(reader.readline())
                if not token:
                    break

          ### joonho.lim @ 2019-03-15
                if token.find('n_iters=') == 0 or token.find('max_length=') == 0 :

                    continue
                token = token.split('\t')[0].strip('_')

                token = token.strip()
                vocab[token] = index
                index += 1
        self.vocab = vocab
        
    def convert_tokens_to_ids(self, tokens):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            try:
                ids.append(self.vocab[token])
            except:
                ids.append(1)
        if len(ids) > 10000:
            raise ValueError(
                "Token indices sequence length is longer than the specified maximum "
                " sequence length for this BERT model ({} > {}). Running this"
                " sequence through BERT will result in indexing errors".format(len(ids), 10000)
            )
        return ids


def _lazy_dataset_loader(pt_file):
    dataset = pt_file    
    yield dataset
    
def News_to_input(text, openapi_key, tokenizer):
    newstemp = do_lang(openapi_key, text)
    news = newstemp.split(' ./SF ')[:-1]
    
    bertdata = BertData(tokenizer)
    sent_labels = [0] * len(news)
    tmp = bertdata.preprocess(news)
    if(not tmp): return None
    #print(tmp)
    b_data_dict = {"src":tmp[0],
               "tgt": [0],
               "labels":[0,0,0],
               "src_sent_labels":sent_labels,
               "segs":tmp[2],
               "clss":tmp[3],
               "src_txt":tmp[4],
               "tgt_txt":'hehe'}
    b_list = []
    b_list.append(b_data_dict) 
    return b_list

## 추출요약 테스트

In [316]:
openapi_key = '9318dc23-24ac-4b59-a99e-a29ec170bf02'
chat_history = "뉴스 "
user_input = '''
일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다.
정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다.
앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다.
한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다.
가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다.
그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다.
가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다.
그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다.
'''
vocab = get_kobert_vocab()
tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)


bot_input_ids = News_to_input(chat_history + user_input, openapi_key, tokenizer)
print('------------------------------------------------------------')


#logger.info('Loading checkpoint from %s' % test_from)
checkpoint = torch.load('../models/bert_classifier/model_step_1000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
model.load_cp(checkpoint)
model.eval()  

chat_history_ids = summary(args, bot_input_ids, 0, '', None, model)
print(chat_history_ids)
print('------------------------------------------------------------')
print(chat_history_ids[0])
pred_lst = list(chat_history_ids[0][:3])
print(pred_lst)
print('------------------------------------------------------------')
final_text = ''
for i,a in enumerate(user_input.split('. ')):
    if i in pred_lst:
        print(i, a)
        final_text = final_text+a+'. '
chat_history = user_input + '<token>' +final_text
print('------------------------------------------------------------')
print(final_text)
print('------------------------------------------------------------')

using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
------------------------------------------------------------


[2023-01-22 09:11:50,492 INFO] loading archive file ../001_bert_morp_pytorch
[2023-01-22 09:11:50,494 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



[[0 7 3 1 4 5 2 6]]
------------------------------------------------------------
[0 7 3 1 4 5 2 6]
[0, 7, 3]
------------------------------------------------------------
0 
일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다.
정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다.
앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다.
한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다.
가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다.
그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다.
가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다.
그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다.

------------------------------------------------------------

일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다.
정부 대변인인 가토 가쓰노부(加

In [ ]:
def get_top_sentences(user_input, model, tokenizer):
    bot_input_ids = News_to_input(user_input, openapi_key, tokenizer)
    
    chat_history_ids = summary(args, bot_input_ids, 0, '', None, model)
    pred_lst = list(chat_history_ids[0])
    #print(pred_lst)
    final_text = []
    for p in pred_lst:
        if(p < len(user_input.split('.'))):
            final_text.append((p, user_input.split('.')[p]))
    return final_text
    

checkpoint = torch.load('../models/bert_classifier/model_step_1000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
model.load_cp(checkpoint)
model.eval()  
print(get_top_sentences(user_input, model, tokenizer))

[2023-01-22 09:11:58,695 INFO] loading archive file ../001_bert_morp_pytorch
[2023-01-22 09:11:58,697 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



[(0, '\n일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다'), (1, '\n정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다'), (7, '\n그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다'), (3, '\n한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다'), (5, '\n그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다'), (4, '\n가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다'), (6, '\n가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다'), (2, '\n앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다')]


In [109]:
checkpoint = torch.load('../models/bert_classifier2/model_step_35000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
model.load_cp(checkpoint)
model.eval()  
print(get_top_sentences(user_input, model, tokenizer))

[2023-01-21 08:36:57,581 INFO] loading archive file ../001_bert_morp_pytorch
[2023-01-21 08:36:57,583 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



[(6, '\n가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다'), (4, '\n가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다'), (2, '\n앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다'), (3, '\n한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다'), (0, '\n일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다'), (1, '\n정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다'), (5, '\n그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다'), (7, '\n그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다')]


In [11]:
def add_topk_to_df(df, model, tokenizer):
    start = time.time()
    topk = []
    for i,context in enumerate(df['context']):
        text = eval(context)
        top = None
        top = get_topk_sentences(len(text)//4+1, ' '.join(text), model, tokenizer)
        if(top):
            topk.append(top)
        else:
            topk.append('Hello world')
            
    df['topk'] = topk
    df = df.drop(df[df['topk'] == 'Hello world'].index)
    print(f"{time.time()-start:.4f} sec")
    return df

[(1, '\n정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다. '), (4, '\n가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다. '), (5, '\n그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다. '), (7, '\n그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다. ')]


In [12]:
# 공백 정리
import re, unicodedata
from string import whitespace, punctuation
from pykospacing import Spacing
from nltk import word_tokenize, sent_tokenize
import pandas as pd

def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)

    # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()
    return result


def text_filter(text): # str -> 전처리 -> 문장 배열
    text = clean_byline(text)
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣.,]+')
    exclusions = exclude_pattern.findall(text)
    result = exclude_pattern.sub(' ', text).strip()
    spacing = Spacing()
    kospacing_txt = spacing(result) 
    sentences = sent_tokenize(kospacing_txt) 
    return sentences

def make_df(name, model):
    start = time.time()
    df = pd.read_csv(f'../{name}.csv')
    text = []
    for i,context in enumerate(df['context']):
        preprocess = text_filter(context)[:60]
        if(len(preprocess) > 3):
            text.append([(i,v) for i,v in enumerate(preprocess)])
        else:
            text.append('Hello world')
            
    df['text'] = text
    df = df.drop(df[df['text'] == 'Hello world'].index)
    print(f"{time.time()-start:.4f} sec")
    return df


2023-01-21 08:04:42.140146: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-21 08:04:42.140195: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ef17d8c6010c
2023-01-21 08:04:42.140206: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ef17d8c6010c
2023-01-21 08:04:42.141025: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 450.80.2
2023-01-21 08:04:42.141049: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 450.80.2
2023-01-21 08:04:42.141055: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 450.80.2
2023-01-21 08:04:42.146978: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instruct

## Get Topic

In [13]:
import pandas as pd

all_df = pd.read_csv('../result(paraphrase).csv')
all_df.head()

,Unnamed: 0,title,originallink,link,description,pubDate,Topic,Top_n_words
0,0,"검찰, <b>삼성전자</b> 자회사 핵심기술 국외유출범 5명 기소",https://view.asiae.co.kr/article/2023011518041...,https://n.news.naver.com/mnews/article/277/000...,<b>삼성전자</b> 자회사인 세메스가 세계 최초로 개발한 &apos;초임계 반도체...,"Mon, 16 Jan 2023 12:13:00 +0900",24,메스 - 기술 - 자회사 - 유출 - 세정 - 장비 - 기소 - 지검 - 공판 - 수원
1,1,"<b>삼성전자</b>, 강신봉 前 요기요 대표 영입…온라인 강화 목표",https://economist.co.kr/article/view/ecn202301...,https://n.news.naver.com/mnews/article/243/000...,<b>삼성전자</b>가 강신봉 전 요기요 대표를 온라인 세일즈 담당 부사장으로 영입...,"Mon, 16 Jan 2023 21:27:00 +0900",27,영입 - 요기 - 강신봉 - 대표 - 온라인 - 부사장 - 세일즈 - 담당 - 조직...
2,2,"<b>삼성전자</b>, ‘e식품관’에서 새해맞이 프로모션 진행",https://www.mk.co.kr/article/10606058,https://n.news.naver.com/mnews/article/009/000...,<b>삼성전자</b>가 자사 제품과 식품을 연계해 합리적이고 스마트한 쇼핑 경험을 ...,"Mon, 16 Jan 2023 10:03:00 +0900",7,식품 - 새해 - 기획 - 닷컴 - 다양 - 자사 - 연계 - 맞이 - 합리 - 프로모션
3,3,&quot;땡큐 <b>삼성전자</b>&quot;…미국 텍사스주에 생긴 &apos;<...,https://www.hankyung.com/economy/article/20230...,https://n.news.naver.com/mnews/article/015/000...,15일 산업계에 따르면 지난해 12월 미국 텍사스주 윌리엄슨카운티는 <b>삼성전자<...,"Sun, 15 Jan 2023 14:01:00 +0900",18,테일러 - 공장 - 완공 - 경계현 - 텍사스 - 파운드리 - 고속도로 - 사장 -...
4,4,매출 세계 1위 대만 TSMC까지 &quot;투자 축소&quot;…이제 남은 건 <...,https://hankookilbo.com/News/Read/A20230115142...,https://n.news.naver.com/mnews/article/469/000...,반도체 위탁생산(파운드리) 업계 1위인 대만의 TSMC가 올해 경기 악화로 설비 투...,"Tue, 17 Jan 2023 04:31:00 +0900",5,tsmc - 매출 - 반도체 - 대만 - 분기 - 세계 - 파운드리 - 위탁 - 생...


In [14]:
from newspaper import Article

def context(x):
    try:
        article = Article(x, language='ko')
        article.download()
        article.parse()
        return article.text
    except:
        return None

In [15]:
all_df['Topic']

0      24
1      27
2       7
3      18
4       5
       ..
884     3
885     2
886    19
887     2
888    28
Name: Topic, Length: 889, dtype: int64

In [16]:
topics = []
for i in sorted(all_df['Topic'].unique()):
    if(i != -1):
        print(i, len(all_df[all_df['Topic'] == i]))
        topics.append(all_df[all_df['Topic'] == i])
    
topics[0].head(2)

0 68
1 68
2 67
3 43
4 42
5 37
6 32
7 32
8 32
9 29
10 25
11 25
12 24
13 22
14 21
15 20
16 20
17 19
18 19
19 17
20 16
21 15
22 15
23 15
24 14
25 13
26 12
27 11
28 10


,Unnamed: 0,title,originallink,link,description,pubDate,Topic,Top_n_words
74,90,<b>삼성전자</b> 의료기기 위상 추락…생산실적 &apos;4위→10위&apos;,https://www.dailymedi.com/news/news_view.php?w...,https://www.dailymedi.com/news/news_view.php?w...,﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿<b>삼성전자</b> 2021년 의료기기 생산 실적이 883억원으...,"Tue, 17 Jan 2023 06:02:00 +0900",0,영업 - 이익 - 분기 - 69 - 전년 - 감소 - 지난해 - 대비 - 잠정 - ...
121,139,“<b>삼성전자</b> 실적 충격이 일종의 바닥 신호”,https://weekly.donga.com/3/all/11/3890689/1,https://n.news.naver.com/mnews/article/037/000...,<b>삼성전자</b>는 1월 6일 지난해 4분기 영업이익이 4조3000억 원으로 전...,"Sat, 14 Jan 2023 10:02:00 +0900",0,영업 - 이익 - 분기 - 69 - 전년 - 감소 - 지난해 - 대비 - 잠정 - ...


In [68]:
topic_df = topics[10]

In [69]:
len(topic_df)

25

### 전처리

In [18]:
for s in topic_df['title']:
    print(s)

[주간 추천주] 대형주 러브콜…<b>삼성전자</b>·LG엔솔 &apos;관심&apos;
<b>삼성전자</b>와 중소기업 협업처럼…&apos;소부장&apos; 평가지원에 300억 투입
<b>삼성전자</b>, 반도체·AI·차세대통신 R&amp;D &apos;올인&apos;… 새 사업 기회 찾는다
<b>삼성전자</b> 미래는 &apos;전장사업&apos;…하만과 시너지 키운다
[특징주]아남<b>전자</b>, <b>삼성</b> 하만-페라리 차세대 차량용 솔루션 개발 파트너사 선...
<b>삼성전자</b>, 레인보우로보틱스와 전방위 기술 협력 나서
&apos;<b>삼성전자</b>에 갑질&apos; 브로드컴…200억원 상생기금 낸다
<b>삼성전자</b>, 인도 정부와 &apos;관세 분쟁&apos;…2천600억원 면제 관련 이견
한송네오텍 자회사, <b>삼성전자</b>가 투자한 美 &apos;에너베이트&apos;와 맞손
<b>삼성전자</b>·인텔·퀄컴, 올해 반도체 칩 설계 AI에 3억 달러 투자
&apos;<b>삼성전자</b> 갑질 혐의&apos; 브로드컴, 자진 시정방안 최종안 마련
<b>삼성전자</b> 투과율 88% EUV 펠리클 개발…핵심부품 내재화
<b>삼성전자</b>, 인도 정부와 2600억 규모 &apos;관세 분쟁&apos;
공정위, ‘<b>삼성전자</b> 갑질’ 브로드컴에 “부품 계약 강제 금지…상생기금 200...
[공시줍줍]<b>삼성전자</b> 유통파트너 &apos;미래반도체&apos; 증권신고서 읽어보기
<b>삼성전자</b>, 반도체 1조 신규 투자하면 세금 2500억 감면
[단독] <b>삼성전자</b> 승진제도 파격실험…학벌보다 전문가 우대
정부, <b>삼성전자</b> 반도체 1조 투자하면 ‘800억→1500억’ 세금 감면 확대키로
브로드컴, <b>삼성전자</b>에 대한 갑질 의혹 ‘사실상 인정’…자정안 공개
<b>삼성전자</b>·포스코·BGF리테일, 중기부 &apos;이달의 상생볼&apos; 선정
&apos;<b>삼성전자</b>에 갑질&apos;

In [19]:
topic_df.to_csv('../paraphrase_topic20.csv')

In [20]:
topic_df.head(2)

,Unnamed: 0,title,originallink,link,description,pubDate,Topic,Top_n_words
77,93,[주간 추천주] 대형주 러브콜…<b>삼성전자</b>·LG엔솔 &apos;관심&apos;,http://www.inews24.com/view/1558322,https://n.news.naver.com/mnews/article/031/000...,"유안타증권은 <b>삼성전자</b>, <b>삼성</b>엔지니어링 HK이노엔을 추천주로...","Sun, 15 Jan 2023 12:01:00 +0900",10,브로드컴 - 인도 - 상생 - 반도체 - 200 - 관세 - euv - 혐의 - 투...
81,98,<b>삼성전자</b>와 중소기업 협업처럼…&apos;소부장&apos; 평가지원에 3...,https://www.etoday.co.kr/news/view/2212669,https://www.etoday.co.kr/news/view/2212669,이투데이=세종=박준상 기자 | 소부장 7개 분야 100개 과제 평가 검증 지원 반도...,"Sun, 15 Jan 2023 11:02:00 +0900",10,브로드컴 - 인도 - 상생 - 반도체 - 200 - 관세 - euv - 혐의 - 투...


## Preprocess

In [66]:
# 공백 정리
import re, unicodedata
from string import whitespace, punctuation
from pykospacing import Spacing
from nltk import word_tokenize, sent_tokenize
import pandas as pd
from tqdm import tqdm
from newspaper import Article

def context(x):
    try:
        article = Article(x, language='ko')
        article.download()
        article.parse()
        return article.text
    except:
        return None


def add_context_to_df(df):
    contexts = []
    for i, link in enumerate(tqdm(df['link'])):
        cont = context(link)
        if(cont):
            contexts.append(cont)
        else:
            contexts.append('delete this')
    df['text'] = contexts
    df = df.drop(df[df['text'] == 'delete this'].index)
    return df


def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    pattern_tag = re.compile('(<([^>]+)>)|(&([^>]+);)')
    result = pattern_tag.sub('', text)
    result = pattern_email.sub('', result)
    result = pattern_url.sub('', result)

    # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()
    return result


def text_filter(text): # str -> 전처리 -> 문장 배열
    text = clean_byline(text)
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣.,]+')
    exclusions = exclude_pattern.findall(text)
    result = exclude_pattern.sub(' ', text).strip()
    spacing = Spacing()
    kospacing_txt = spacing(result) 
    sentences = sent_tokenize(kospacing_txt) 
    return sentences


def text_filter2(text): # 제목, description 전처리
    text = clean_byline(text)
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣.,]+')
    exclusions = exclude_pattern.findall(text)
    result = exclude_pattern.sub(' ', text).strip()
    return result


def extract_context(df:pd.DataFrame) -> pd.DataFrame:
    df = add_context_to_df(df)

    start = time.time()
    text = []
    for i,context in enumerate(df['text']):
        preprocess = text_filter(context)[:60]
        if(len(preprocess) > 3):
            text.append([v for i,v in enumerate(preprocess)])
        else:
            text.append('Hello world')
            
    df['context'] = text
    df = df.drop(df[df['context'] == 'Hello world'].index)
    df = df.drop(columns=['text'])
    df['title'] = df['title'].apply(text_filter2)
    df['description'] = df['description'].apply(text_filter2)
    #df = df.drop(columns= ['Unnamed: 0'])
    print(f"{time.time()-start:.4f} sec")
    return df

In [70]:
topic_context_df = extract_context(topic_df)
len(topic_context_df)

100%|██████████| 25/25 [00:06<00:00,  4.00it/s]
/tmp/ipykernel_22728/4237231848.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = contexts


11.1166 sec


25

In [71]:
topic_context_df = topic_context_df.drop(columns= ['Unnamed: 0'])
topic_context_df.head(2)

,title,originallink,link,description,pubDate,Topic,Top_n_words,context
77,대형주 러브콜 삼성전자 LG엔솔,http://www.inews24.com/view/1558322,https://n.news.naver.com/mnews/article/031/000...,"유안타증권은 삼성전자, 삼성엔지니어링 HK이노엔을 추천주로 꼽았다. 삼성전자에 대해...","Sun, 15 Jan 2023 12:01:00 +0900",10,브로드컴 - 인도 - 상생 - 반도체 - 200 - 관세 - euv - 혐의 - 투...,[국내 증권사들이 다음 주 시장에서 대형주에 관심 가질 것을 조언했다.15일 금융투...
81,삼성전자와 중소기업 협업처럼 평가지원에 300억 투입,https://www.etoday.co.kr/news/view/2212669,https://www.etoday.co.kr/news/view/2212669,이투데이 세종 박준상 기자 소부장 7개 분야 100개 과제 평가 검증 지원 반도...,"Sun, 15 Jan 2023 11:02:00 +0900",10,브로드컴 - 인도 - 상생 - 반도체 - 200 - 관세 - euv - 혐의 - 투...,[소 부장 7개 분야 100개 과 제 평가 검증 지원 15일 정부는 2023년도 소...


In [146]:
from tqdm import tqdm 

checkpoint = torch.load('../models/bert_classifier2/model_step_35000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
model.load_cp(checkpoint)
model.eval()  

def add_topk_to_df(df, model, tokenizer):
    start = time.time()
    topk = []
    for i,context in enumerate(tqdm(df['context'])):
        context = eval(context)
        top = None
        if(len(context) > 3):
            top = get_top_sentences(' '.join(context), model, tokenizer)
        if(top):
            topk.append(top)
        else:
            topk.append('Hello world')
            
    df['topk'] = topk
    df = df.drop(df[df['topk'] == 'Hello world'].index)
    print(f"{time.time()-start:.4f} sec")
    return df

[2023-01-21 08:45:10,696 INFO] loading archive file ../001_bert_morp_pytorch
[2023-01-21 08:45:10,697 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



In [147]:
df = add_summary_to_df(topic_context_df, model, tokenizer)

100%|██████████| 25/25 [00:32<00:00,  1.30s/it]

32.3974 sec


In [322]:
df = pd.read_csv('../paraphrase_topic20_pre.csv')
df = df.drop(columns= ['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.2','Unnamed: 0.4', 'Unnamed: 0.5'])
df = df.drop(columns= ['Unnamed: 0.3', 'Unnamed: 0.6'])
df.head(2)

,title,originallink,link,description,pubDate,Topic,Top_n_words,context,topk
0,대형주 러브콜 삼성전자 LG엔솔,http://www.inews24.com/view/1558322,https://n.news.naver.com/mnews/article/031/000...,"유안타증권은 삼성전자, 삼성엔지니어링 HK이노엔을 추천주로 꼽았다. 삼성전자에 대해...","Sun, 15 Jan 2023 12:01:00 +0900",10,브로드컴 - 인도 - 상생 - 반도체 - 200 - 관세 - euv - 혐의 - 투...,['국내 증권사들이 다음 주 시장에서 대형주에 관심 가질 것을 조언했다.15일 금융...,"[(13, ' HK이노엔에 대해선 타이신 짠 케이캡 중국 명 의 중국 NRDL 포..."
1,삼성전자와 중소기업 협업처럼 평가지원에 300억 투입,https://www.etoday.co.kr/news/view/2212669,https://www.etoday.co.kr/news/view/2212669,이투데이 세종 박준상 기자 소부장 7개 분야 100개 과제 평가 검증 지원 반도...,"Sun, 15 Jan 2023 11:02:00 +0900",10,브로드컴 - 인도 - 상생 - 반도체 - 200 - 관세 - euv - 혐의 - 투...,['소 부장 7개 분야 100개 과 제 평가 검증 지원 15일 정부는 2023년도 ...,"[(14, ' 산업부는 반도체, 디스플레이협회 등 업종별 주요 협 단체의 네트워크를..."


In [324]:
df.to_csv('../paraphrase_topic20_pre.csv')

In [325]:
topic_context_df = pd.read_csv('../paraphrase_topic20_pre.csv')
topic_context_df.head(2)

,Unnamed: 0,title,originallink,link,description,pubDate,Topic,Top_n_words,context,topk
0,0,대형주 러브콜 삼성전자 LG엔솔,http://www.inews24.com/view/1558322,https://n.news.naver.com/mnews/article/031/000...,"유안타증권은 삼성전자, 삼성엔지니어링 HK이노엔을 추천주로 꼽았다. 삼성전자에 대해...","Sun, 15 Jan 2023 12:01:00 +0900",10,브로드컴 - 인도 - 상생 - 반도체 - 200 - 관세 - euv - 혐의 - 투...,['국내 증권사들이 다음 주 시장에서 대형주에 관심 가질 것을 조언했다.15일 금융...,"[(13, ' HK이노엔에 대해선 타이신 짠 케이캡 중국 명 의 중국 NRDL 포..."
1,1,삼성전자와 중소기업 협업처럼 평가지원에 300억 투입,https://www.etoday.co.kr/news/view/2212669,https://www.etoday.co.kr/news/view/2212669,이투데이 세종 박준상 기자 소부장 7개 분야 100개 과제 평가 검증 지원 반도...,"Sun, 15 Jan 2023 11:02:00 +0900",10,브로드컴 - 인도 - 상생 - 반도체 - 200 - 관세 - euv - 혐의 - 투...,['소 부장 7개 분야 100개 과 제 평가 검증 지원 15일 정부는 2023년도 ...,"[(14, ' 산업부는 반도체, 디스플레이협회 등 업종별 주요 협 단체의 네트워크를..."


In [156]:
i = 1
df['topk'][i]
#get_top_sentences(''.join(eval(df['context'][i])), model, tokenizer)

[(14,
  ' 산업부는 반도체, 디스플레이협회 등 업종별 주요 협 단체의 네트워크를 적극적으로 활용해 수요, 공급기업의 매칭을 활성화하 고 우수 협력사례를 선정하고 홍보해 우수기업의 참여를 확대할 계획 이라고 밝혔다'),
 (8, ' 올해 사업부터는 원자재와 재료비 가격 상승 등을 반영해  비용 지원 상한을 2억 원에서 3억 원으로 확대했다'),
 (6,
  ' 이 사업은 국내 중소 중견기업이 개발한 소부장 제품의 성능이 대기업 등 수요기업의 생산 설비에서 평가하고 검증할 수 있도록 지원하는 사업이다'),
 (4, ' 에이티아이와 삼성전자의 협업이 가능했던 이유는 정부의 소재 부품 장비 소부장 양산 성능 평가 지원사업 덕이다'),
 (13, ' 생산 속도는 3배 늘렸고, 양산성도 확보해 매출액은 12억 원에 달했다'),
 (5,
  ' 15일 정부는 2023년도 소부장 양산 성능 평가지원 사업을 공고하고 7개 분야 100개 과제에 총 300억 원의 국비를 투자할 계 획이라고 밝혔다'),
 (2, ' 반도체 공정에 필요한 얇은 원판 뒷 면의 반사율과 투과율, 두께 측정을 접촉하지 않고도 할 수 있는 장비다'),
 (7,
  ' 2019년 추경 사업으로 반도체와 디스플레이 분야에 최초 지원 한 후 4년간 498개 수요, 공급 기업 성능평가에 1470억 원을 지원했다'),
 (10,
  ' 지원 대상도 대일 100대 고의존도 품목에서 반도체와 디스플레이 등 7개 분야 150대 핵심 전략 기술 관련 품목으로 개편해 지원한다'),
 (3, ' 이 기업의 매출액은 58억 원에 달한다'),
 (11, ' 양산 성능 평가 대상 품목에 대한 수요기업의 구매 의향서와 동의서를 제출하면 먼저 지원해 사업화 성과를 높일 계획이다'),
 (9, ' 추가 성능 개선이 필요하면 2억 원을 추가로 지원한다'),
 (0, '소 부장 7개 분야 100개 과 제 평가 검증 지원 15일 정부는 2023년도 소부장 양산 성능 평가지원 사업을 공고했다'),
 (1,
  ' 사진제공 산업통상자원부

# df['topk'][i]

In [157]:
eval(str(df['topk'][6]))[:5]

[(12, '공정위는 오는 10일부터 다음 달 18일까지 40일간  이해관계인 및 관계부처의 의견을 수렴할 계획이다'),
 (11, ' 3년간 기술 지원도 제공한다'),
 (5,
  '이번 잠정 동의의결안에는 부품 공급계약 강제 및 부품 선택권 제한 금지 등 경쟁질서 회복 방안과 200억원 규모의 반도체 분야 중소사업자 상생 방안, 삼성전자가 구매한 부품에 대한 기술지원 품질보증 등이 포함됐다'),
 (3,
  ' 앞서 공정위는 브로드컴이 삼성전자에 RF프론트 엔드, 와이파이, 블루투스, 위성 항법 시스템 GNSS 등 스마트 기기의 핵심 부품 공급에 관해 3년간 장기 계약을 강제한 혐의를 조사해왔다'),
 (4,
  ' 해당 계약에는 지난 2021년 1월부터 2023년 말까지 삼성전자가 브로드컴의 부품을 매년 7억6천만 달러 약 9천5 00억원 이상을 구매하고, 실제 구매 금액이 미달하면 차액을 배상한다는 내용이 담겼다')]

## K-means 클러스터링

## 토픽 요약하기

In [158]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string
from sklearn.cluster import KMeans


def get_clustered_df(sentences, clusternum):
    print(clusternum)
    document_df = pd.DataFrame()
    document_df['opinion_text'] = [str(t) for t in sentences]
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

    lemmar = WordNetLemmatizer()
    
    # 토큰화한 각 단어들의 원형들을 리스트로 담아서 반환
    def LemTokens(tokens):
        return [lemmar.lemmatize(token) for token in tokens]

    # 텍스트를 Input으로 넣어서 토큰화시키고 토큰화된 단어들의 원형들을 리스트로 담아 반환
    def LemNormalize(text):
        # .translate인자에 구두점 dict넣어주어서 구두점 삭제해준 상태로 토큰화시키기!
        return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
    
    tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                            ngram_range=(1,2),
                            min_df=0.05, max_df=0.85)

    # fit_transform으로 위에서 구축한 도구로 텍스트 벡터화
    ftr_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
    # K-means로 3개 군집으로 문서 군집화시키기

    kmeans = KMeans(n_clusters=clusternum, max_iter=10000, random_state=42)
    # 비지도 학습이니 feature로만 학습시키고 예측
    cluster_label = kmeans.fit_predict(ftr_vect)
    
    # 군집화한 레이블값들을 document_df 에 추가하기
    document_df['cluster_label'] = cluster_label
    return document_df.sort_values(by=['cluster_label'])
    

def get_topic_sentences(df, clabel):
    lst = []
    for i,t in enumerate(df[df['cluster_label'] == clabel]['opinion_text']):
        print(i, t)
        lst.append(t)
    return lst 

In [159]:
df = pd.read_csv('../paraphrase_topic20_pre.csv')
len(df)

25

In [193]:
def delete_similar(input_sentences):
    sorted_sentences = sorted(input_sentences, key=lambda x:x[1][::-1])
    prev_len = len(sorted_sentences)
    
    for i in range(5):
        prev = sorted_sentences[0]
        processed_sentences = []
        for j,sentence in enumerate(sorted_sentences[1:]):
            s1 = set(prev[1].split())
            s2 = set(sentence[1].split())
            actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
            if(actual_jaccard < 0.5): # if not similar
                processed_sentences.append(prev)
                prev = sentence
            else:
                print(prev)
                print(sentence)
                print(actual_jaccard)
                print('-------------------------------------------')
                
        s1 = set(prev[1].split())
        s2 = set(sentence[1].split())
        if(actual_jaccard < 0.5): # if not similar
            processed_sentences.append(prev)
        
        sorted_sentences = sorted(processed_sentences, key=lambda x:x[1])
        print(prev_len, len(sorted_sentences))
        
        if(prev_len == len(sorted_sentences)):
            break
        prev_len = len(sorted_sentences)
        
    return sorted_sentences


def get_first_topk_sentences(name):
    df = pd.read_csv(f'../{name}.csv')

    first_sentences = []
    topk_sentences = []
    topk_percent = 30
    for a,b in zip(df['context'], df['topk']):
        context = eval(str(a))
        topk = eval(str(b))
        k = int(len(topk)*(topk_percent/100))
        topk = topk[:k]
        
        first = []
        for item in topk:
            if(item[0] == 0): 
                first.append(item)
                
        if(len(first) == 0):
            first_sentences += [(0, context[0])]
            topk_sentences += topk
        else:
            first_sentences += first
            topk.remove(first[0])
            topk_sentences += topk
                
    print('before delete similar:', len(first_sentences), len(topk_sentences))
    first_sentences = delete_similar(first_sentences)
    topk_sentences = delete_similar(topk_sentences)
    print('after delete similar:', len(first_sentences), len(topk_sentences))
    return first_sentences, topk_sentences

first_sentences, topk_sentences = get_first_topk_sentences('paraphrase_topic20_pre')

before delete similar: 25 87
25 25
87 87
after delete similar: 25 87


In [194]:
topk_sentences = sorted(topk_sentences, key=lambda x:x[1])
topk_sentences

[(11, ' 3년간 기술 지원도 제공한다'),
 (6, ' C랩에 선정된 기업들에 1년간 최대 1억원의 사업화 지원금을 지급하고, 기업의 성장 단계에 맞는 맞춤형 컨설팅을 지원한다'),
 (13, ' HK이노엔에 대해선 타이신 짠 케이캡  중국 명 의 중국 NRDL 포함 이후 처방이 확대될 것으로 예상된다'),
 (14, ' MS는 첨단 운전자 지원 시스템 등 차량 소프트웨어와 메타버스를 활용한 차량 구입의 미래를 제시했다'),
 (4,
  ' 계약에는 2021년 1월부터 2023년 말까지 삼성전자가 브로드컴의 부품을 매년 7억 6,000만 달러어치 이상 구매해야 하고, 구매 금액이 계약 액수에 미치지 못하면 차액을 브로드컴에 배 상한다는 내용이 담겼습니다'),
 (6, ' 고객이 어떤 디바이스를 쓰더라도 같은 경험을 느끼고 차원이 다른 가치를 느낄 수 있게 할 계획이다'),
 (6, ' 공정위가 조사에 착수하자 브로드컴 측은 동의의결을 신청했고, 지난해 9월 공정위가 이를 받아들이면서 관련 절차가 시작됐다'),
 (13, ' 구글은 음성 명령으로 차량 제어 구동이  가능한 안드로이드 오토 체험 공간을 마련했다'),
 (10, ' 국내 반도체 유통시장에는 다양한 유통기업이 존재하는데요'),
 (11,
  ' 그래픽 제공 한국 딜로이트 삼성전자 인텔 퀄컴, 올해 반도체 칩 설계 AI에 3억 달러 투자 한국 딜로이트 그룹은 AI 인공지능 기반 반도체 설계 시장을 조명하는 내용의 차세대 반도체, 인류에 새로운 세상 열어준 다 리포트 국문 본을 발간했다고 13일 밝혔다'),
 (4,
  ' 기존 승객은 엘리베이터 탑승 중 특이사항이 발생하면 비상통화장치를 이용해 신고하지만, 이 같은 경우 음성으로만 상황이 전달돼 내용이 왜곡될 가능성이 있었다'),
 (9, ' 대학 원 생 등 취업 희망자 재직자 대상으로 5년간 총 750명 반도체 전문인력을 양성하는 것이 목표다'),
 (4,
  ' 동의의결은 공정위 조사 심의를 받는 사업자가 스스로 원상회복, 피해 구제 등 시 정

In [305]:
from itertools import combinations

def delete_repet_str(user_text):
    
    text = user_text.split()
    x = len(text)
    comb = list(combinations(range(x), 2))
    sorted_comb = sorted(comb, key=lambda x: x[1]-x[0], reverse = True)
    for item in sorted_comb:
        start, end = item
        if(end-start <= len(sorted_comb) and end-start>1):
            find_str = ' '.join(text[start:end])
            rest_str = ' '.join(text[end:])
            idx = rest_str.find(find_str)
            if idx != -1:
                print('deleted :', find_str)
                ret = ' '.join(text[:end]) + ' ' + rest_str[:idx] + rest_str[idx+len(find_str)+1:]
                return ret
    return user_text


text1 = '''중소벤처기업부는 지난해 11월 이달의 상생 볼로 삼성전자, 포스코 인터내셔널, 포스코, BGF 리테일의 4개사의 상생 협력 활동을 선정했다고 5일 밝혔으며 
중소벤처기업부는 지난해 11월 이달의 상생 볼로 삼성전자, 포스코 인터내셔널, 포스코, BGF 리테일의 4개사의 상생 협력 활동을 선정했다고 5일 밝혔다.'''
delete_repet_str(text1)

deleted : 중소벤처기업부는 지난해 11월 이달의 상생 볼로 삼성전자, 포스코 인터내셔널, 포스코, BGF 리테일의 4개사의 상생 협력 활동을 선정했다고 5일


'중소벤처기업부는 지난해 11월 이달의 상생 볼로 삼성전자, 포스코 인터내셔널, 포스코, BGF 리테일의 4개사의 상생 협력 활동을 선정했다고 5일 밝혔으며 밝혔다.'

In [291]:
import time
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import random


def summarize_topic(document_df, topic_num, tokenizer, model):
    sentences = []
    numbers = []
    for i,t in enumerate(document_df[document_df['cluster_label'] == topic_num]['opinion_text']):
        sentences.append(eval(t)[1])
        numbers.append(eval(t)[0])
    result = make_summarization(sentences, tokenizer, model)
    avg = sum(numbers)/len(numbers)
    return (avg, result)


def make_summarization(sentences, tokenizer, model):
    if(len(sentences) < 4): return max(sentences, key=lambda x:len(x))
    split = []
    for i in range(len(sentences)//8):
        split.append(sentences[:8])
        sentences = sentences[8:]

    for i in range(len(split)):
        if(len(sentences) == 0): break
        split[i].append(sentences.pop())
    
    if(len(sentences) != 0): split.append(sentences)
    
    split_sum = []
    for sentences in split:
        text = '\n'.join(sentences)
        start = time.time()
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

        summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=256, min_length=48,  eos_token_id=1)
        print(f"{time.time()-start:.4f} sec")
        sum_result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        print(sum_result)
        split_sum.append(sum_result)
        print(len(split), len(split_sum))
        print('-----------------------------------------------')
    
    if(len(split_sum)==1):
        return split_sum[0]
      
    return ' '.join(split_sum)

In [292]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
    
def summarize_first_sentences(processed_sentences, tokenizer, model):
    clusternum = 1
    document_df = get_clustered_df(processed_sentences, clusternum)
    sum_result = []
    for c in range(clusternum):
        temp = get_topic_sentences(document_df, c)
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        summ = summarize_topic(document_df, c, tokenizer, model)
        sum_result.append(summ)
        print(summ)
        print('===================================================')
    
    first_result = (sum_result[0][0], max(sum_result[0][1].split('. '), key=lambda x:len(x)))
    return first_result
    
def summarize_topk_sentences(processed_sentences, tokenizer, model):
    clusternum = len(processed_sentences)//7
    document_df = get_clustered_df(processed_sentences, clusternum)
    sum_result = []
    for c in range(clusternum):
        temp = get_topic_sentences(document_df, c)
        print('---------------------------------------------------')
        summ = summarize_topic(document_df, c, tokenizer, model)
        sum_result.append(summ)
        print(summ)
        print('***************************************************')
        
    return sorted(sum_result, key= lambda x: x[0])

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [293]:
sum_result1 = summarize_first_sentences(first_sentences, tokenizer, model)
sum_result2 = summarize_topk_sentences(topk_sentences, tokenizer, model)

1
0 (0, '2023년 기업공개 IPO 시장이 본격적으로 열린 가운데 반도체 유통 전문기업가 코스닥 시장의 문을 두드리고 있어요.')
1 (0, '인도 세무정보국이 삼성전자 현지법인을 수입 관세를 회피한 혐의로 기소했다.')
2 (0, '소 부장 7개 분야 100개 과 제 평가 검증 지원 15일 정부는 2023년도 소부장 양산 성능 평가지원 사업을 공고했다.')
3 (0, '세종시 어진 동 정부세종청사 공정거래위원회.')
4 (0, '서울 뉴스1 이 민주 기자 중소벤처기업부는 지난해 11월 이달의 상생 볼 로 삼성전자, 포스코 인터내셔널, 포스코, BGF 리테일의 4개사의 상생 협력 활동을 선정했다고 5일 밝혔다.이달의 상생 볼은 매월 기업의 우수 상생협력 활동을 포상해 상생협력 활동을 격려하고 동반성장 문화를 확산하는 행사다.')
5 (0, '삼성전자에 갑질 한 혐의를 받는 미국 반도체 기업 브로드컴이 200억원 규모의 상생기금을 조성하기로 했다.')
6 (0, '삼성전자가 인도 정부와 2600억 원 규모의 관세 면제 관련 분쟁을 벌이고 있는 것으로 알려졌습니다.')
7 (0, '삼성전자가 올해부터 직원의 승진 평가에 특화평가 를 도입합니다.')
8 (0, '삼성전자가 극자외선 EUV 노광 공정 핵심 부품으로 꼽히는 펠리클 의 국산화에 다가섰다.')
9 (0, '삼성전자 제공 연합뉴스 자료사진.')
10 (0, '삼성전자 자회사 하만이 이탈리아 글로벌 완성차 업체 페라리의 차세대 자동차 솔루션 파트너사로 선정됐다는 소식에 아남전자의 주가가 장 중 강세를 보이고 있다.16일 오후 1시 52분 현재 아남 전자는 유가 증권시장에서 전 거래일 대비 % 오른 2010원에 거래되고 있다.')
11 (0, '추경호 경제 부총리 겸 기획재정부 장관이 3일 오후 서울 중구 롯데호텔에서 열린 범 금융 신년 인 사회에 참석해 신년사를 하고 있다.')
12 (0, '삼성전자 등 국내 제 조사에 갑질을 한 혐의로 공정거래위원회의 조사를 받은 미국 반도체 기업 브로드컴이 200억

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


2.1120 sec
중소벤처기업부는 지난해 11월 이달의 상생 볼로 삼성전자, 포스코 인터내셔널, 포스코, BGF 리테일의 4개사의 상생 협력 활동을 선정했다고 5일 밝혔으며 중소벤처기업부는 지난해 11월 이달의 상생 볼로 삼성전자, 포스코 인터내셔널, 포스코, BGF 리테일의 4개사의 상생 협력 활동을 선정했다고 5일 밝혔다.
3 1
-----------------------------------------------
1.6365 sec
삼성전자가 극자외선 EUV 노광 공정 핵심 부품으로 꼽히는 펠리클 의 국산화에 다가언더삼성전자 자회사 하만이 이탈리아 글로벌 완성차 업체 페라리의 차세대 자동차 솔루션 파트너사로 선정됐다는 소식에 아남전자의 주가가 장 중 강세를 보이고 있다.
3 2
-----------------------------------------------
1.7730 sec
15일 금융투자증권업계에 따르면 SK증권은 LG에너지솔루션, 삼성전기, SK스퀘어를 추천하였으며, 미국 반도체 기업 브로드컴이 삼성전자 등 국내 스마트 기기 제조사에 불공정한 계약을 강제하지 않겠다는 자정안을 마련했다며 구체적 내용을 오늘 9일 공개하고 의견을 수렴하기로 했다.
3 3
-----------------------------------------------
(0.0, '중소벤처기업부는 지난해 11월 이달의 상생 볼로 삼성전자, 포스코 인터내셔널, 포스코, BGF 리테일의 4개사의 상생 협력 활동을 선정했다고 5일 밝혔으며 중소벤처기업부는 지난해 11월 이달의 상생 볼로 삼성전자, 포스코 인터내셔널, 포스코, BGF 리테일의 4개사의 상생 협력 활동을 선정했다고 5일 밝혔다. 삼성전자가 극자외선 EUV 노광 공정 핵심 부품으로 꼽히는 펠리클 의 국산화에 다가언더삼성전자 자회사 하만이 이탈리아 글로벌 완성차 업체 페라리의 차세대 자동차 솔루션 파트너사로 선정됐다는 소식에 아남전자의 주가가 장 중 강세를 보이고 있다. 15일 금융투자증권업계에 따르면 SK증권은 LG에너

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


5.0387 sec
지난해 12월 30일 윤석 열 대통령이 윤석 열 대통령이 반도체 등 국가전략산업에 대한 세제 지원을 확대하는 방안을 검토하라고 지시한 뒤 나흘 만에 확정 판결을 내린 가운데, 브로드컴은 삼성전자에 스마트폰, 태블릿PC 등 스마트 기기 핵심 부품인 RF 프런트 엔드 RFFE, 와이파이 Wi Fi , GNSS 위성항법 시스템 등을 납품하면서 구매 주문을 승인하지 않고 선적을 중단하는 등 불공정한 수단으로 장기 계약을 강제한 혐의로 공정위로부터 직권조사를 받아왔으며 반도체 등 국가전략산업에 대한 세제 지원을 확대하는 방안을 검토하라 고 지시한 뒤 나흘 만에 확정 판결을 내렸으며, 산업부는 반도체, 디스플레이협회 등 업종별 주요 협 단체의 네트워크를 적극적으로 활용해 수요, 공급기업의 매칭을 활성화하 고 우수 협력사례를 선정하고 홍보해 우수 협력사례를 선정하고 홍보해 우수기업의 참여를 확대할 계획 이라고 밝혔다.
2 1
-----------------------------------------------
1.4378 sec
지난해 3월 이전 출시된 스마트 기기다 지난해 3월 이전 출시된 스마트 기기다 지난해 3월 이전 출시된 스마트 기기다 지난해 3월 이전 출시된 스마트 기기다 지난해 3월 이전 출시된 스마트 기기다 지난해 3월 이전 출시된 스마트 기기다 지난해 3월 이전 출시된 스마트 기기다
2 2
-----------------------------------------------
(10.8, '지난해 12월 30일 윤석 열 대통령이 윤석 열 대통령이 반도체 등 국가전략산업에 대한 세제 지원을 확대하는 방안을 검토하라고 지시한 뒤 나흘 만에 확정 판결을 내린 가운데, 브로드컴은 삼성전자에 스마트폰, 태블릿PC 등 스마트 기기 핵심 부품인 RF 프런트 엔드 RFFE, 와이파이 Wi Fi , GNSS 위성항법 시스템 등을 납품하면서 구매 주문을 승인하지 않고 선적을 중단하는 등 불공정한 수단으로 장기 계약을 강제한 혐의로 공정위로부터 직권조사를 받

In [214]:
sum_result1

(0.0,
 '15일 금융투자증권업계에 따르면 SK증권은 LG에너지솔루션, 삼성전기, SK스퀘어를 추천하였으며, 미국 반도체 기업 브로드컴이 삼성전자 등 국내 스마트 기기 제조사에 불공정한 계약을 강제하지 않겠다는 자정안을 마련했다며 구체적 내용을 오늘 9일 공개하고 의견을 수렴하기로 했다.')

In [215]:
sum_result2

[(4.0,
  '올해 반도체 등 국가전략기술 분야 신규 시설에 자금을 투입하는 대기업은 투자액의 25%만큼 세금을 깎을 수 있게 될 전망이다.'),
 (6.0,
  'C랩에 선정된 기업들에 1년간 최대 1억원의 사업화 지원금을 지급하고, 기업의 성장 단계에 맞는 맞춤형 컨설팅을 지원한 노동당 칩 설계 과정에서 발생하는 경우의 수는 체스 10123 와 바둑 10360 보다 각각 731배, 250배 높은 1090,000에 달했다.'),
 (6.25,
  '삼성전자는 앞으로 30일 이내에 세무조사국에 관련 소명서를 제출해야 하며, 세무조사국은 통지서에서 관세 면 제 관련 차액이 납부되지 않은 이유와 관계자가 문책당하지 않은 점도 지적했다.'),
 (8.0,
  '삼성전자는 가전 부문에서 초연결 로 미래 먹거리를 구상하고 전장 부문에서는 하만과 협업을 내세워 자동차용 통합 시스템 반도체 SoC, 자동차용 통합 시스템 반도체 SoC, 인공지능 AI 기 술 등을 활용한 전 장 기술 통합으로 수익성을 높일 계획이다.'),
 (9.035714285714286,
  '추경호 부총리 겸 기재부 장관은 이 날 브리핑에서 반도체는 작년 수출의 %, 설비 투자의 %를 차지한 우리 경제의 중추 산업으로 한국의 미래 경쟁력 및 국가 안보, 생존과 직결되는 전략 자산 이라며 이번 조치가 우리 반도체 산업의 초격차  확보와 재도약을 위한 마중물이 되기를 기대한 다 고 밝혔다. 브로드컴은 구매 주문 승인 중단 등을 활용, 삼성전자에 스마트 기기  부품 공급과 관련해 3년 간 장기 계약 LTA 체결을 강제한 혐의로 공정위 조사 대상에 올랐으나, 야당 반대로 법인세를 양껏 내리지 못한 정부가 대신 세액공제를 기업 투자 촉진 유인책으로 들고 나오면서 미리뷰는 고객 신고 접수와 동시에 현대엘리베이터 고객 케어센터에서 영상으로 현장을 모니터링해, 능동적으로 상황에 대처할 수 있도록 독려했다. 삼성전자가 올해 반도체 생산시설에 1조원을 투자한다면 현재 세금 감면액은 800억원에 그치지만, 정부안 기준

In [216]:
final_result = [v for i,v in [sum_result1] + sum_result2]
final_sentences = [s if '.' in s[-2: ] else s+'. ' for i,s in enumerate(final_result)]
final_sentences

['15일 금융투자증권업계에 따르면 SK증권은 LG에너지솔루션, 삼성전기, SK스퀘어를 추천하였으며, 미국 반도체 기업 브로드컴이 삼성전자 등 국내 스마트 기기 제조사에 불공정한 계약을 강제하지 않겠다는 자정안을 마련했다며 구체적 내용을 오늘 9일 공개하고 의견을 수렴하기로 했다.',
 '올해 반도체 등 국가전략기술 분야 신규 시설에 자금을 투입하는 대기업은 투자액의 25%만큼 세금을 깎을 수 있게 될 전망이다.',
 'C랩에 선정된 기업들에 1년간 최대 1억원의 사업화 지원금을 지급하고, 기업의 성장 단계에 맞는 맞춤형 컨설팅을 지원한 노동당 칩 설계 과정에서 발생하는 경우의 수는 체스 10123 와 바둑 10360 보다 각각 731배, 250배 높은 1090,000에 달했다.',
 '삼성전자는 앞으로 30일 이내에 세무조사국에 관련 소명서를 제출해야 하며, 세무조사국은 통지서에서 관세 면 제 관련 차액이 납부되지 않은 이유와 관계자가 문책당하지 않은 점도 지적했다.',
 '삼성전자는 가전 부문에서 초연결 로 미래 먹거리를 구상하고 전장 부문에서는 하만과 협업을 내세워 자동차용 통합 시스템 반도체 SoC, 자동차용 통합 시스템 반도체 SoC, 인공지능 AI 기 술 등을 활용한 전 장 기술 통합으로 수익성을 높일 계획이다.',
 '추경호 부총리 겸 기재부 장관은 이 날 브리핑에서 반도체는 작년 수출의 %, 설비 투자의 %를 차지한 우리 경제의 중추 산업으로 한국의 미래 경쟁력 및 국가 안보, 생존과 직결되는 전략 자산 이라며 이번 조치가 우리 반도체 산업의 초격차  확보와 재도약을 위한 마중물이 되기를 기대한 다 고 밝혔다. 브로드컴은 구매 주문 승인 중단 등을 활용, 삼성전자에 스마트 기기  부품 공급과 관련해 3년 간 장기 계약 LTA 체결을 강제한 혐의로 공정위 조사 대상에 올랐으나, 야당 반대로 법인세를 양껏 내리지 못한 정부가 대신 세액공제를 기업 투자 촉진 유인책으로 들고 나오면서 미리뷰는 고객 신고 접수와 동시에 현대엘리베이터 고객 케어센터에서 영

In [309]:
new_final_sentences = []
for s in final_sentences:
    new = delete_repet_str(s)
    new_final_sentences.append(delete_repet_str(s))
    if(new != s):
        print(s)
        print('---------------------------------')
        print(new)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
deleted : 자동차용 통합 시스템 반도체 SoC,
deleted : 자동차용 통합 시스템 반도체 SoC,
삼성전자는 가전 부문에서 초연결 로 미래 먹거리를 구상하고 전장 부문에서는 하만과 협업을 내세워 자동차용 통합 시스템 반도체 SoC, 자동차용 통합 시스템 반도체 SoC, 인공지능 AI 기 술 등을 활용한 전 장 기술 통합으로 수익성을 높일 계획이다.
---------------------------------
삼성전자는 가전 부문에서 초연결 로 미래 먹거리를 구상하고 전장 부문에서는 하만과 협업을 내세워 자동차용 통합 시스템 반도체 SoC, 인공지능 AI 기 술 등을 활용한 전 장 기술 통합으로 수익성을 높일 계획이다.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
deleted : 올해 반도체 생산시설에 1조원을 투자한다면 현재 세금 감면액은 800억원에 그치지만, 정부안 기준으로는 1500억원에 달하는 세금을 감면 받을 수 있게 되었으며,
deleted : 올해 반도체 생산시설에 1조원을 투자한다면 현재 세금 감면액은 800억원에 그치지만, 정부안 기준으로는 1500억원에 달하는 세금을 감면 받을 수 있게 되었으며,
추경호 부총리 겸 기재부 장관은 이 날 브리핑에서 반도체는 작년 수출의 %, 설비 투자의 %를 차지한 우리 경제의 중추 산업으로 한국의 미래 경쟁력 및 국가 안보, 생존과 직결되는 전략 자산 이라며 이번 조치가 우리 반도체 산업의 초격차  확보와 재도약을 위한 마중물이 되기를 기대한 다 고 밝혔다. 브로드컴은 구매 주문 승인 중단 등을 활용, 삼성전자에 스마트 기기  부품 공급과 관련해 3년 간 장기 계약 LTA 체결을 강제한 혐의로 공정위 조사 대상에 올랐으나, 야당 반

In [310]:
new_final_sentences

['15일 금융투자증권업계에 따르면 SK증권은 LG에너지솔루션, 삼성전기, SK스퀘어를 추천하였으며, 미국 반도체 기업 브로드컴이 삼성전자 등 국내 스마트 기기 제조사에 불공정한 계약을 강제하지 않겠다는 자정안을 마련했다며 구체적 내용을 오늘 9일 공개하고 의견을 수렴하기로 했다.',
 '올해 반도체 등 국가전략기술 분야 신규 시설에 자금을 투입하는 대기업은 투자액의 25%만큼 세금을 깎을 수 있게 될 전망이다.',
 'C랩에 선정된 기업들에 1년간 최대 1억원의 사업화 지원금을 지급하고, 기업의 성장 단계에 맞는 맞춤형 컨설팅을 지원한 노동당 칩 설계 과정에서 발생하는 경우의 수는 체스 10123 와 바둑 10360 보다 각각 731배, 250배 높은 1090,000에 달했다.',
 '삼성전자는 앞으로 30일 이내에 세무조사국에 관련 소명서를 제출해야 하며, 세무조사국은 통지서에서 관세 면 제 관련 차액이 납부되지 않은 이유와 관계자가 문책당하지 않은 점도 지적했다.',
 '삼성전자는 가전 부문에서 초연결 로 미래 먹거리를 구상하고 전장 부문에서는 하만과 협업을 내세워 자동차용 통합 시스템 반도체 SoC, 인공지능 AI 기 술 등을 활용한 전 장 기술 통합으로 수익성을 높일 계획이다.',
 '추경호 부총리 겸 기재부 장관은 이 날 브리핑에서 반도체는 작년 수출의 %, 설비 투자의 %를 차지한 우리 경제의 중추 산업으로 한국의 미래 경쟁력 및 국가 안보, 생존과 직결되는 전략 자산 이라며 이번 조치가 우리 반도체 산업의 초격차 확보와 재도약을 위한 마중물이 되기를 기대한 다 고 밝혔다. 브로드컴은 구매 주문 승인 중단 등을 활용, 삼성전자에 스마트 기기 부품 공급과 관련해 3년 간 장기 계약 LTA 체결을 강제한 혐의로 공정위 조사 대상에 올랐으나, 야당 반대로 법인세를 양껏 내리지 못한 정부가 대신 세액공제를 기업 투자 촉진 유인책으로 들고 나오면서 미리뷰는 고객 신고 접수와 동시에 현대엘리베이터 고객 케어센터에서 영상으로 현장을 모니터링해, 능동적으로 상황

In [80]:
clusternum = len(final_sentences)//7 + 1
final_cluster_df = get_clustered_df(final_sentences, clusternum)
final_cluster_df

1


/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,opinion_text,cluster_label
0,전자랜드가 2023년을 맞아 삼성전자에서 진행하는 대규모 할인 행사 삼성전자 005...,0
1,"(1, '여러 가지 가전제품을 구매하면 풍성한 혜택을 제공하는 다 품목 할인 행사도...",0
2,"(2, '이 달 13일부터 16일까지 45개의 전자랜드 행사점에서 1300만 원 ...",0
3,"(3, '삼성전자는 행사 기간 동안 프리미엄 가전제품을 두 가지 이상 구매한 고객들...",0
4,"(4, '1월 16일부터 31일까지는 삼세권 인증샷 이벤트 를 진행해 삼성전자 세일...",0
5,"(5, '스웨덴 국민들의가 총파업에 들어 갈 경우 국민들의 일반적 시선은 그들의 당...",0


In [82]:
len(final_result)

6

In [163]:
checkpoint = torch.load('../models/bert_classifier2/model_step_35000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model2 = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)

[2023-01-19 06:23:45,517 INFO] loading archive file ../001_bert_morp_pytorch
[2023-01-19 06:23:45,519 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



In [174]:
#checkpoint = torch.load('../models/bert_classifier2/model_step_35000.pt', map_location=lambda storage, loc: storage)
#config = BertConfig.from_json_file(args.bert_config_path)
#model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
def make_summarization_final(sentences, model):
    if(len(sentences) < 4): return max(sentences, key=lambda x:len(x))
    split = []
    for i in range(len(sentences)//8):
        split.append(sentences[:8])
        sentences = sentences[8:]

    for i in range(len(split)):
        if(len(sentences) == 0): break
        split[i].append(sentences.pop())
    
    if(len(sentences) != 0): split.append(sentences)
    
    split_sum = []
    for sentences in split:
        text = '\n'.join(sentences)
        start = time.time()
        sent = get_topk_sentences(len(sentences)//2, ' '.join(sentences), model) 
        print(f"{time.time()-start:.4f} sec")
        print(sent)
        split_sum += sent
        print(len(split), len(split_sum))
        print('-----------------------------------------------')
    
    if(len(split_sum)==1):
        return split_sum[0]
      
    return split_sum


In [175]:
final_sum = make_summarization_final(final_result, model2)
final_sum

using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
1.6871 sec
[(3, '   TSMC는 지난해 같은 기간 대비 매출을 43% 가량 늘리며 세계 반도체 매출 1위는 유지한 것으로 추정되나, TSMC는 지난해 3분기에 이어 4분기에도 세계 반도체 매출 1위는 유지한 것으로 추정된다. '), (4, '  세계 1위 파운드리업체인 TSMC 매출에서 데이터 서버 등에 사용되는 고성능 컴퓨터용 반도체가 1위에 올라섰다는 점은 세계 파운드리 시장의 판도 변화를 보여주는 중요한 의미가 있다. '), (5, '  \n웨이저자 TSMC CEO 는 12일 콘퍼런스콜을 통해 3나노 미세공정에서 고성능 컴퓨터용 반도체 고객사 수요가 공급 능력을 웃도는 상황을 보이고 있다며, 2023년 하반기부터 매출에 상당한 부분을 기여하게 될 것 이라고 밝혔다. '), (8, ' 마이크론은 인력 구조조정에도 나섰으며 전년 대비 각각 %, % 감소한 규모인 매출 증가율은 10%로 집계됐다. ')]
3 4
-----------------------------------------------
using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached mod

[(3,
  '   TSMC는 지난해 같은 기간 대비 매출을 43% 가량 늘리며 세계 반도체 매출 1위는 유지한 것으로 추정되나, TSMC는 지난해 3분기에 이어 4분기에도 세계 반도체 매출 1위는 유지한 것으로 추정된다. '),
 (4,
  '  세계 1위 파운드리업체인 TSMC 매출에서 데이터 서버 등에 사용되는 고성능 컴퓨터용 반도체가 1위에 올라섰다는 점은 세계 파운드리 시장의 판도 변화를 보여주는 중요한 의미가 있다. '),
 (5,
  '  \n웨이저자 TSMC CEO 는 12일 콘퍼런스콜을 통해 3나노 미세공정에서 고성능 컴퓨터용 반도체 고객사 수요가 공급 능력을 웃도는 상황을 보이고 있다며, 2023년 하반기부터 매출에 상당한 부분을 기여하게 될 것 이라고 밝혔다. '),
 (8, ' 마이크론은 인력 구조조정에도 나섰으며 전년 대비 각각 %, % 감소한 규모인 매출 증가율은 10%로 집계됐다. '),
 (2, '                                                                   . '),
 (3,
  '  세계적인 경기 침체 우려 속에 글로벌 반도체 기업인 삼성전자와 TSMC의 실적 온도 차의 원인을 파운드리 사업의 수 주 경쟁력에서 찾고 있는 가운데, 삼성전자는 파운드리 업체가 보유한 공정 기술과 특허를 반영해 반도체를 설계하기 때문에 경기 부진의 여파를 상대적으로 비껴 간 것으로 보인다. '),
 (4,
  ' 삼성전자는 메모 메모리 반도체와 파운드리를 모두 운영하는 사업자인데 메모리 반도체 업황 침체로 분 기 매출 1위를  되찾지 못하였으며 이는 고금리 영향으로 글로벌 IT 기업들이 투자를 축소하고 스마트폰과 PC 등의 수요가 줄어들자 직격탄을 맞았기 때문이다. '),
 (7,
  '   삼성전자는 오는 31일 2022년 4분기 및 연간 실적을 발표하는 가운데 시장에선 반도 체 부문 매출액을 19조 3천억 원 수준으로 보고 있으며 삼성전자는 잠정 실적 발표를 통해 2 022년 4분기 매출 

## 생성 요약

In [193]:
import time
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import random

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

def make_summarization(sentences):
    text = '\n'.join(sentences)
    start = time.time()
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

    summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512, min_length=64,  eos_token_id=1)
    print(f"{time.time()-start:.4f} sec")
    sum_result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    return sum_result

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [194]:
import time
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import random

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

def summarize_topic(document_df, topic_num):
    sentences = []
    numbers = []
    for i,t in enumerate(document_df[document_df['cluster_label'] == topic_num]['opinion_text']):
        sentences.append(eval(t)[1])
        numbers.append(eval(t)[0])
    result = make_summarization(sentences)
    avg = sum(numbers)/len(numbers)
    return (avg, result)

def make_summarization(sentences, tokenizer, model):
    if(len(sentences) < 4): return max(sentences, key=lambda x:len(x))
    split = []
    for i in range(len(sentences)//8):
        split.append(sentences[:8])
        sentences = sentences[8:]

    for i in range(len(split)):
        if(len(sentences) == 0): break
        split[i].append(sentences.pop())
    
    if(len(sentences) != 0): split.append(sentences)
    
    split_sum = []
    for sentences in split:
        text = '\n'.join(sentences)
        start = time.time()
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

        summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512, min_length=48,  eos_token_id=1)
        print(f"{time.time()-start:.4f} sec")
        sum_result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        print(sum_result)
        split_sum.append(sum_result)
        print(len(split), len(split_sum))
        print('-----------------------------------------------')
    
    if(len(split_sum)==1):
        return split_sum[0]
      
    return ' '.join(split_sum)

In [195]:
document_df

,opinion_text,cluster_label
150,"(5, ' 특히 최근 주목받고 있는 자동차용 반도체, 인공지능 AI , 사물인터넷 ...",0
71,"(5, ' 비즈니스 포스트 글로벌 경제팀에서 연재하는 삼성의 라이벌 기획은 삼성전자...",0
148,"(5, ' 특히 이 반도체는 단독이 아닌 CPU와 함께 쓰여 CPU 업체로부터 인증...",0
8,"(5, ' 6 삼성전자 파운드리 분사 준비됐나, 고객사와 경쟁 한계 비즈니스포스트 ...",0
137,"(5, ' 지난해 363억달러 보다 시설 투자액을 최대 % 줄이겠다 고 선언한 것이...",0
...,...,...
121,"(12, ' 인도네시아는 오랜 기간 테슬라 공장 유치를 추진해 왔습니다. ')",24
11,"(12, ' DDR 뒤에 붓는 숫자가 높을수록 차세대 제품이며, 숫자가 높아질 때마...",24
58,"(12, ' 미국 텍 사스주 테일 러시에 건설 중인 삼성전자 신규 파운드리 공장. ')",24
174,"(12, '장중 머우 창업주의 표현은 다소 거칠었지만, 그가 세계화 및 자유무역 ...",24


In [196]:
def summarize_topic(document_df, topic_num):
    sentences = []
    numbers = []
    for i,t in enumerate(document_df[document_df['cluster_label'] == topic_num]['opinion_text']):
        sentences.append(eval(t)[1])
        numbers.append(eval(t)[0])
    result = make_summarization(sentences)
    avg = sum(numbers)/len(numbers)
    return (avg, result)

In [125]:
result = summarize_topic(document_df, 1)
result

1.3943 sec
대만 T운드리 위탁 생산 글로벌 1위인 대만 TSMC가 지난해 3분기에 이어 4분기에도 삼성전자를 꺾고 세계 반도체 매출 1위에 오른 소식에도, 하반기 반도체 업황 반등 기대감이 작용해 삼성전자 주가는 소폭 상승하고 있다.  
2 1
-----------------------------------------------
1.2878 sec
TSMC는 지난해 같은 기간 대비 매출을 43% 가량 늘리며 세계 반도체 매출 1위는 유지한 것으로 추정되나, TSMC는 지난해 3분기에 이어 4분기에도 세계 반도체 매출 1위는 유지한 것으로 추정된다. 
2 2
-----------------------------------------------


(7.363636363636363,
 '대만 T운드리 위탁 생산 글로벌 1위인 대만 TSMC가 지난해 3분기에 이어 4분기에도 삼성전자를 꺾고 세계 반도체 매출 1위에 오른 소식에도, 하반기 반도체 업황 반등 기대감이 작용해 삼성전자 주가는 소폭 상승하고 있다.   TSMC는 지난해 같은 기간 대비 매출을 43% 가량 늘리며 세계 반도체 매출 1위는 유지한 것으로 추정되나, TSMC는 지난해 3분기에 이어 4분기에도 세계 반도체 매출 1위는 유지한 것으로 추정된다. ')

In [137]:
sentences = []
for i,t in enumerate(document_df[document_df['cluster_label'] == 6]['opinion_text']):
    print(i, t)
    sentences.append(eval(t)[1])

0 (5, ' 하지만 올해는 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출시하지 않을 예정이다. ')
1 (6, ' 딜 사이트 김민기 기자 삼성전자의 새로운 먹거리로 보였던  무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있다. ')
2 (7, ' 신제품 대신 업데이트로 대응 업계에서는 올해 삼성전자의 무선 이어폰 신제품 출시는 없을 것으로 전망하고 있다. ')
3 (15, ' 하지만 이번에는 소프트웨어 업데이트 이외에  새로운 신제품 출시는 이뤄지지 않는다. ')
4 (16, ' 삼성전자 관계자는 갤럭시 버즈 2 신제품 나온 지가 6개월 밖에 안 된 상황 이라며 올해 신제품 출시와 관련해서는 아직 잘 모르겠다 고 답했다. ')
5 (15, ' 해당 기능은 원 UI 소프트웨어 지원 스마트폰과 연동해 이 용할 수 있다. ')


In [138]:
result = make_summarization(sentences)
result

1.7708 sec


'삼성전자의 새로운 먹거리로 보였던 무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있는 가운데, 삼성전자는 올해 무선 이어폰 신제품 출시는 없을 것으로 전망하고 업데이트로 대응 업계에서는 올해 무선 이어폰 신제품 출시는 없을 것으로 전망하고 있다.     '

In [140]:
def summarize_topic(document_df, topic_num):
    sentences = []
    for i,t in enumerate(document_df[document_df['cluster_label'] == topic_num]['opinion_text']):
        sentences.append(eval(t)[1])
    result = make_summarization(sentences)
    return result

In [139]:
result = summarize_topic(document_df, 0)
result

1.7528 sec


'더 풍부하고 섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공하는 LE 오디오 기술은 보다 풍부하고 섬세한 사운드, 섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공하는 것이 특징이라고 회사는 설명했다.         '

In [141]:
result1 = summarize_topic(document_df, 1)
result1

2.8699 sec


'삼성전자는 갤럭시 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤럭시 버즈 2 프로와 갤럭시 Z 플립4 폴드4의 소프트웨어 업데이트를 시행하여 사용자가 직접 듣는 그대로의 생생한 사운드를 녹음할 수 있다고 밝혔다.  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [142]:
result2 = summarize_topic(document_df, 2)
result2

2.3841 sec


'삼성전자는 다음 워치5와 워치 4 시리즈에 카메라 줌 인, 줌 아웃 기능을 추가한다고 밝히며 다음 달 진행되는 갤럭시 언팩에서 새로운 갤럭시 스마트폰과 관련된 더 많은 내용을 소개할 것 이라고 말했다  \n 삼성전자는 다음 달 갤럭시 워치5와 워치 4 시리즈에 카메라 줌 인, 줌 아웃 기능을 추가한다고 전했다.'

In [143]:
result3 = summarize_topic(document_df, 3)
result3

2.5630 sec


'스마트폰을 먼 거리에 세워두고 단체 사진이나 단독 셀카를 찍을 때 카메라가 있는 곳까지 돌아갈 필요 없이 손목에서 바로 손쉽게 화면 배율 조정을 할 수 있게 되어 손목에서 바로 손쉽게 화면 배율 조정을 할 수 있게 되었으며,  \n 스마트폰을 먼 거리에 세워두고 단체 사진이나 단독 셀카를 찍을 때 카메라가 있는 곳까지 돌아갈 필요 없이 손목에서 바로 손쉽게 화면 배율 조정이 가능해진다.'

In [144]:
result4 = summarize_topic(document_df, 4)
result4

2.1447 sec


'360 오디오 레코딩은 차세대 블 루투스 오디오 기술 표 준인 LE 오디오가 적용됐다.  련련련련련련련련련련련련련련련련련련은 차세대 블루투스 오디오 기술 표 준인 LE 오디오를 적용했다.'

In [145]:
result5 = summarize_topic(document_df, 5)
result5

2.9359 sec


'전문 장비 없이도 누구나  언제 어디서든 고품질의 실감 나는 오디오를 동영상에 담을 수 있다는 게 삼성전자의 설명으로 사용자는 시계 화면을 손가락으로 늘리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있게 되었으며 사용자는 시계 화면을 손가락으로 늘 리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있게 되었다.'